# 0.&nbsp;들어가며

## 0-1. 사전안내
- 텍스트 셀에 써있는 지침 반드시 잘 읽고 따르기
- 코드 셀의 주석 부분은 궁금하면 참고로 읽어보기

## 0-2. 데이터 준비
- roboflow에서 아래와 같은 형식으로 데이터를 다운받아 드라이브에 옮기기  
  - 훈련 데이터: Other - Tensorflow TFRecord
  - 테스트 데이터: XML - Pascar VOC

In [1]:
from google.colab import drive
drive.mount('/content/drive')

NameError: name 'drive' is not defined

# 1.&nbsp;환경 설정 및 라이브러리 설치

## 1-1. TensorFlow Object Detection API 설치

In [2]:
# TensorFlow 모델 레포지토리를 GitHub에서 클론
# (TensorFlow Object Detection API 설치를 위해 필요)
!pip uninstall Cython -y  # "No module named 'object_detection'" 오류를 임시로 해결하기 위해 Cython을 제거
!git clone --depth 1 https://github.com/tensorflow/models  # TensorFlow 모델 소스 코드를 가져옴 (최신 커밋만 클론)

Found existing installation: Cython 3.0.11
Uninstalling Cython-3.0.11:
  Successfully uninstalled Cython-3.0.11
Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3315/3315), done.
remote: Total 4305 (delta 1211), reused 2190 (delta 917), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.16 MiB | 24.00 MiB/s, done.
Resolving deltas: 100% (1211/1211), done.


In [3]:
# 모델 설정 파일을 models/research 폴더로 복사 및 프로토콜 버퍼 파일 컴파일

# Bash 셸 명령어 실행
%%bash
cd models/research/

# Object Detection API에서 사용하는 .proto 파일을 컴파일하여 Python 코드로 변환
protoc object_detection/protos/*.proto --python_out=.

In [4]:
# setup.py 파일 수정: TensorFlow 모델 레포지토리를 TF v2.8.0에 맞춰 설치
import re  # 정규 표현식을 사용하기 위한 모듈

# 기존 setup.py 파일의 내용을 읽어옴
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()  # setup.py 파일 내용을 문자열로 읽기

# 수정된 내용을 새로운 setup.py 파일에 작성
with open('/content/models/research/setup.py', 'w') as f:
    # "tf-models-official>=2.5.1" 문자열을 "tf-models-official==2.8.0"으로 변경
    # TensorFlow 버전 호환성을 위해 특정 버전으로 고정
    s = re.sub('tf-models-official>=2.5.1', 'tf-models-official==2.8.0', s)
    f.write(s)  # 수정된 내용을 새로운 setup.py 파일에 저장


- 아래 셀 실행 시 발생하는 **의존성 오류는 무시**
  
- 중간에 '**세션 다시 시작**' 팝업이 뜰 경우  
세션 다시 시작 클릭 -> 아래 셀을 한 번 더 실행

In [1]:
# Object Detection API 설치 (참고: 이 코드 블록 실행에 약 10분 정도 소요될 수 있음)

# PyYAML 라이브러리 관련 문제 해결
# Colab 환경에서 PyYAML v5.4.1을 설치할 수 없으므로 PyYAML v5.3 버전을 설치
!pip install pyyaml==5.3

# Object Detection API를 설치
# 연구(research) 폴더에 있는 Python 패키지를 설치하여 사용할 준비
!pip install /content/models/research/

# TensorFlow 버전을 2.8.0으로 다운그레이드
# Colab 환경에서 TensorFlow v2.10과의 호환성 문제(2022년 10월 기준)가 있어 v2.8.0으로 변경
!pip install tensorflow==2.8.0

# TensorFlow v2.8.0과 호환되는 CUDA v11.0 설치
# TensorFlow v2.8.0은 CUDA v11.0과 호환되므로 이를 설치하여 GPU 가속을 지원
!pip install tensorflow_io==0.23.1  # TensorFlow와 호환되는 I/O 라이브러리 설치

# # CUDA v11.0 관련 패키지를 다운로드하고 설치
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
# !mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600

# # CUDA v11.0 설치 파일을 다운로드
# !wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

# # CUDA 설치 파일을 dpkg 명령어로 설치
# !dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb

# # CUDA의 공개 키를 추가하여 패키지를 인증
# !apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub

# # 시스템 패키지 업데이트 및 CUDA 툴킷 v11.0 설치
# !apt-get update && sudo apt-get install cuda-toolkit-11-0

# # CUDA 라이브러리 경로를 환경 변수에 추가
# !export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

Processing ./models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697355 sha256=98fe06999707faa13e25e5e175a2bf57b117b0cc148f04be97d48430a701ef60
  Stored in directory: /tmp/pip-ephem-wheel-cache-f_vs31k9/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1
  Using cached tensorflow-2.8.0-cp310-cp310-manylinux2010_x86_64.whl.metadata (2.9 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 33.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: t

- 아래 셀 실행 시 발생하는 오류는 무시

In [2]:
# Protobuf 패키지 문제 해결을 위한 설치 과정

!pip uninstall -y protobuf  # Protobuf 패키지를 제거 (-y 옵션으로 사용자 확인 없이 제거).
!pip install protobuf==3.20.1  # Protobuf 버전을 3.20.1로 다시 설치

Found existing installation: protobuf 4.25.5
Uninstalling protobuf-4.25.5:
  Successfully uninstalled protobuf-4.25.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.61.0 requires protobuf!=4.0.*,!=4.21.*,!=4.22.0,!=4.23.*,!=4.24.*,<6.0.0.dev0,>=3.20.3, but you have protobuf 3.20.1 which is incompatible.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.19.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-aiplatform 1.71.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev

- 아래 셀을 통해 강제로 세션 종료  
그 다음 셀로 넘어가서 계속 진행하면 됨

In [ ]:
import os

# 현재 실행 중인 프로세스를 강제로 종료
os.kill(os.getpid(), 9)  # os.getpid()로 현재 프로세스 ID를 가져오고, 신호 9(SIGKILL)로 해당 프로세스를 종료

- 아래 셀 출력 값이 다음과 같다면 성공
```
Name: protobuf  
Version: 3.20.1
```


In [1]:
# Protobuf 패키지 정보 확인
!pip show protobuf

Name: protobuf
Version: 3.20.1
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: apache-beam, google-ai-generativelanguage, google-api-core, google-cloud-aiplatform, google-cloud-bigquery-connection, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-datastore, google-cloud-firestore, google-cloud-functions, google-cloud-iam, google-cloud-language, google-cloud-pubsub, google-cloud-resource-manager, google-cloud-translate, google-generativeai, googleapis-common-protos, grpc-google-iam-v1, grpcio-status, orbax-checkpoint, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata, wandb


## 1-2. TensorFlow Object Detection API 설치 확인

In [2]:
# 모델 빌더 테스트 파일 실행: 설정이 올바르게 작동하는지 확인하기 위한 테스트

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py
# Object Detection API의 모델 빌더 구성 요소를 테스트하는 스크립트를 실행
# 이 스크립트는 설치 및 구성 파일들이 제대로 작동하는지 확인하는 데 사용됨

2024-12-02 02:43:44.188311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:43:44.188348: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7a4cb0283130>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 207, in cb
KeyboardInterrupt: 
2024-12-02 02:43:58.511883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:43:58.511964: W tensorflow/stream_executor/cuda/cuda_driver

# 2.&nbsp;데이터 준비

## 2-1. 라벨 맵(labelmap.txt) 파일 생성
- 테스트 데이터의 형식을 참고하여 아래에 클래스 이름 작성하기

In [3]:
# 객체 탐지 모델이 탐지할 클래스 목록을 포함하는 "labelmap.txt" 파일 생성
%%bash

# labelmap.txt 파일에 클래스 이름을 추가
cat <<EOF >> /content/labelmap.txt
motorcycle
bicycle
kickboard
EOF

## 2-2. 데이터셋 위치 정의(train/test 데이터)

In [4]:
from google.colab import drive

# Google Drive 연결
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 각 파일이 있는 경로 지정
  - `base_dir`
  - `train_record_fname`
  - `val_record_fname`
  - `label_map_pbtxt_fname`

In [17]:
# 작업 디렉토리 설정
base_dir = '/content/drive/MyDrive/data'
train_record_fname = f'{base_dir}/train/-.tfrecord'
val_record_fname = f'{base_dir}/valid/-.tfrecord'
label_map_pbtxt_fname = f'{base_dir}/train/-_label_map.pbtxt'

# 3.&nbsp;훈련 구성 설정

## 3-1. 사용할 모델 설정(SSD MobileNet V2)

In [18]:
# TF2 Object Detection Zoo에서 제공하는 다양한 모델 중 하나를 선택하여 배포할 수 있도록 설정

# 선택한 모델의 이름을 설정
chosen_model = 'ssd-mobilenet-v2-fpnlite-640'

# 사용할 모델들의 구성 정보 딕셔너리
MODELS_CONFIG = {
    # 1. SSD MobileNet V2 (320x320 해상도, COCO 데이터셋 기반)
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',  # 모델 이름
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',  # 기본 파이프라인 구성 파일
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',  # 사전 학습된 체크포인트 파일
    },
    # 2. EfficientDet-D0 (효율적인 작은 크기의 모델, COCO 데이터셋 기반)
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    # 3. SSD MobileNet V2 FPNLite (320x320 해상도, COCO 데이터셋 기반, FPN 사용)
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # 4. CenterNet (현재 작동하지 않음 - 2022년 9월 10일 기준)
    # 'centernet-mobilenet-v2': {
    #     'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #     'base_pipeline_file': 'pipeline.config',
    #     'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    # },
    # 1. SSD MobileNet V2 (640x640 해상도, COCO 데이터셋 기반)
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_640x640_coco17_tpu-8',  # 모델 이름
        'base_pipeline_file': 'ssd_mobilenet_v2_640x640_coco17_tpu-8.config',  # 기본 파이프라인 구성 파일
        'pretrained_checkpoint': 'ssd_mobilenet_v2_640x640_coco17_tpu-8.tar.gz',  # 사전 학습된 체크포인트 파일
    },
    # 2. EfficientDet-D0 (효율적인 작은 크기의 모델, COCO 데이터셋 기반)
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    # 3. SSD MobileNet V2 FPNLite (640x640 해상도, COCO 데이터셋 기반, FPN 사용)
    'ssd-mobilenet-v2-fpnlite-640': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
    },
    # 4. CenterNet (현재 작동하지 않음 - 2022년 9월 10일 기준)
    # 'centernet-mobilenet-v2': {
    #     'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #     'base_pipeline_file': 'pipeline.config',
    #     'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    # }
}

# 선택된 모델에 해당하는 구성 정보 가져오기
model_name = MODELS_CONFIG[chosen_model]['model_name']  # 모델 이름
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']  # 사전 학습된 체크포인트 경로
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']  # 기본 파이프라인 구성 파일 경로

## 3-2. 모델 구성 파일 및 사전 학습된 가중치 다운로드

In [19]:
# "mymodel" 폴더 생성: 사전 학습된 가중치와 구성 파일을 저장할 디렉토리 생성
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# 사전 학습된 모델 가중치 다운로드
import tarfile  # tar 파일 압축 해제를 위한 라이브러리
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

# 설정된 URL에서 가중치 파일 다운로드
!wget {download_tar}

tar = tarfile.open(pretrained_checkpoint)  # 다운로드한 tar 파일 열기
tar.extractall()  # tar 파일의 모든 내용을 현재 디렉토리에 추출
tar.close()  # 파일 닫기

# 학습 구성 파일 다운로드
# 해당 모델의 훈련을 위한 기본 설정이 포함된 구성 파일(pipeline.config)을 다운로드
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file

# 구성 파일을 지정된 URL에서 다운로드
!wget {download_config}

mkdir: cannot create directory ‘/content/models/mymodel/’: File exists
/content/models/mymodel
--2024-12-02 02:52:55--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.63.207, 142.250.31.207, 142.251.111.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.63.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  --.-KB/s    in 0.08s   

2024-12-02 02:52:56 (244 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz.1’ saved [20518283/20518283]

--2024-12-02 02:52:56--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config
Resolving raw.

## 3-3. 훈련 단계 및 배치 크기 설정
$$
\text{Number of epochs} = \frac{\text{num steps} \times \text{batch size}}{\text{number of data}}
$$


In [20]:
# 모델 훈련을 위한 파라미터 설정
num_steps = 125000  # 총 훈련 단계 수 설정 (훈련 반복 횟수)

# 선택한 모델에 따라 배치 크기(batch size) 설정
if chosen_model == 'efficientdet-d0':  # EfficientDet-D0 모델인 경우
  batch_size = 4  # 작은 모델이므로 더 작은 배치 크기 사용
else:
  batch_size = 16

In [21]:
# 파일 경로 및 클래스 수 가져오기
# 모델 훈련에 필요한 설정 파일 및 체크포인트 파일 경로 설정
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file  # 파이프라인 구성 파일 경로
fine_tune_checkpoint = '/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite/ckpt-11.data-00000-of-00001'  # 사전 학습된 체크포인트 파일 경로

# 라벨 맵에서 클래스 수를 계산하는 함수 정의
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util  # Object Detection API에서 제공하는 유틸리티 모듈
    label_map = label_map_util.load_labelmap(pbtxt_fname)  # 라벨 맵 파일 로드
    categories = label_map_util.convert_label_map_to_categories(  # 라벨 맵 데이터를 카테고리로 변환
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)  # 카테고리 인덱스를 생성
    return len(category_index.keys())  # 카테고리(클래스) 수 반환

# 라벨 맵 파일에서 총 클래스 수 계산
num_classes = get_num_classes(label_map_pbtxt_fname)  # 라벨 맵 파일 경로를 전달하여 클래스 수 가져오기
print('Total classes:', num_classes)  # 클래스 수 출력

Total classes: 3


In [22]:
# 구글 드라이브 연결 확인
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 4.&nbsp;사용자 정의 구성 파일 작성

## 4-1. 데이터셋 경로, 라벨 맵 경로, 가중치 경로 등 업데이트

In [23]:
# 기본 파이프라인 파일을 수정하여 사용자 정의 구성 파일 생성
# 데이터셋, 모델 체크포인트, 학습 파라미터를 작성

import re

# 작업 디렉토리 변경
%cd /content/models/mymodel
print('writing custom configuration file')

# 기존 파이프라인 파일 읽기
with open(pipeline_fname) as f:
    s = f.read()

# 수정된 내용을 새로운 구성 파일에 쓰기
with open('pipeline_file.config', 'w') as f:

    # 사전 학습된 체크포인트 경로 설정
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # 학습 데이터(train) 및 검증 데이터(val) 경로 설정
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # 라벨 맵 경로 설정
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # 배치 크기 설정
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # 학습 단계 수(num_steps) 설정
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # 클래스 수(num_classes) 설정
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # 체크포인트 유형을 "classification"에서 "detection"으로 변경
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # SSD MobileNet V2 모델의 경우 학습률(learning rate) 감소
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)
      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # EfficientDet-D0 모델의 경우, TFLite와 호환되지 않는 구성을 수정
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)  # 비율 유지 대신 고정 크기 사용
      s = re.sub('pad_to_max_dimension: true', '', s)  # 패딩 제거
      s = re.sub('min_dimension', 'height', s)  # 최소 크기를 높이로 변경
      s = re.sub('max_dimension', 'width', s)  # 최대 크기를 너비로 변경

    # 수정된 내용을 파일에 작성
    f.write(s)

/content/models/mymodel
writing custom configuration file


In [24]:
# (Optional) 사용자 정의 구성 파일의 내용을 출력하여 확인

!cat /content/models/mymodel/pipeline_file.config
# 터미널에서 pipeline_file.config 파일의 내용을 출력
# 작성된 구성 파일이 올바른지 확인하는 데 사용

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 28.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {

## 4-2. 사용자 정의 파일 경로 설정

In [27]:
# 사용자 정의 구성 파일 경로 및 훈련 체크포인트 저장 디렉토리 설정

pipeline_file = '/content/models/mymodel/pipeline_file.config'  # 사용자 정의 구성 파일 경로
model_dir = '/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite'  # 훈련 중 생성되는 체크포인트 파일을 저장할 디렉토리 경로

# 5.&nbsp;모델 훈련

## 5-1. TensorBoard를 활용한 훈련 모니터링

In [28]:
!pip install tensorboard

In [ ]:
!pip show tensorboard

Name: tensorboard
Version: 2.8.0
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, google-auth, google-auth-oauthlib, grpcio, markdown, numpy, protobuf, requests, setuptools, tensorboard-data-server, tensorboard-plugin-wit, werkzeug, wheel
Required-by: tensorflow


- 아래 코드는 일단 실행  
but 처음엔 아무 내용도 나오지 않음  
모델 학습을 진행하는 중에 새로고침하면 학습 과정 그래프 볼 수 있음

In [ ]:
# TensorBoard 확장 로드 및 로그 디렉토리 설정

# Jupyter/Colab 환경에서 TensorBoard 확장 로드
%load_ext tensorboard

# TensorBoard를 실행하여 '/content/training/train' 디렉토리의 로그 데이터 시각화
%tensorboard --logdir '/content/training/train'

## 5-2. Object Detection API를 이용한 훈련 실행



```
!python /content/models/research/object_detection/model_main_tf2.py \  # Object Detection API의 주요 훈련 스크립트 실행
    --pipeline_config_path={pipeline_file} \  # 사용자 정의 구성 파일의 경로를 전달
    --model_dir={model_dir} \  # 체크포인트 파일과 로그를 저장할 디렉토리 경로 지정
    --alsologtostderr \  # 표준 오류(stdout)로도 로그 메시지를 출력하도록 설정
    --num_train_steps={num_steps} \  # 훈련 단계 수를 설정 (여기서는 {num_steps}로 지정)
    --sample_1_of_n_eval_examples=1  # 평가 데이터에서 1/n 샘플을 사용하여 평가를 수행 (여기서는 모든 예제를 평가)
```



In [ ]:
# 모델 학습 실행
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're us

## 5-3. 훈련 끝난 모델 저장

In [29]:
import os
import re
import numpy as np


output_directory = "/content/drive/MyDrive/moodels/fine_tuned_model"  # 추론 모델 저장 경로

# 최신 체크포인트 찾기
lst = os.listdir(model_dir)
lst = [l for l in lst if 'ckpt-' in l and '.index' in l]  # .index 파일만 필터링
steps = np.array([int(re.findall(r'\d+', l)[0]) for l in lst])  # 체크포인트 번호 추출
latest_checkpoint = f"ckpt-{steps.max()}"  # 가장 큰 번호의 체크포인트 선택
latest_checkpoint_path = os.path.join(model_dir, latest_checkpoint)

print(f"Latest checkpoint: {latest_checkpoint_path}")

# 추론용 모델 저장
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_file} \
    --trained_checkpoint_dir={model_dir} \
    --output_directory={output_directory}


Latest checkpoint: /content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite/ckpt-11
2024-12-02 02:54:10.336461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:54:20.026857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:54:20.026974: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf

In [32]:
import os

# 압축 대상 폴더 경로와 압축 파일 이름 설정
folder_to_zip = '/content/drive/MyDrive/moodels/fine_tuned_model/saved_model'  # 압축할 폴더 경로
output_zip = '/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite/saved_model.zip'  # 생성될 ZIP 파일 경로

# 폴더를 ZIP으로 압축
!zip -r {output_zip} {folder_to_zip}

# 압축 완료 메시지 출력
print(f"Folder '{folder_to_zip}' has been compressed to '{output_zip}'.")

  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/ (stored 0%)
  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/variables/ (stored 0%)
  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/variables/variables.index (deflated 78%)
  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/assets/ (stored 0%)
  adding: content/drive/MyDrive/moodels/fine_tuned_model/saved_model/saved_model.pb (deflated 92%)
Folder '/content/drive/MyDrive/moodels/fine_tuned_model/saved_model' has been compressed to '/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite/saved_model.zip'.


In [31]:
from google.colab import files

# 생성된 ZIP 파일 다운로드
files.download(output_zip)

FileNotFoundError: Cannot find file: /content/saved_model.zip

# 6.&nbsp;TFLite 모델로 변환

## 6-1. 훈련된 모델을 TFLite 모델로 내보내기

In [33]:
# 훈련된 TFLite 모델을 저장할 디렉토리 생성
!mkdir /content/custom_model_lite
output_directory = '/content/drive/MyDrive/models'

# 훈련 디렉토리 경로 설정 (변환 스크립트가 가장 최신 체크포인트 파일을 자동으로 선택)
last_model_path = '/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite'

# TFLite 변환 그래프를 생성
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}


mkdir: cannot create directory ‘/content/custom_model_lite’: File exists
2024-12-02 02:56:57.130563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:57:02.415243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-12-02 02:57:02.415303: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
I1202 02:57:07.676317 137396488953856 api.py:441] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1202 02:57:15.961182 137396488953856 api.py:441] feature_map_spatial_dims: [(80, 80), (40, 40), (2

## 6-2. TensorFlow Lite 모델 저장 및 테스트

In [35]:
# 내보낸 그래프 파일을 TFLite 모델 파일로 변환

import tensorflow as tf

# 저장된 모델(saved_model) 경로를 지정하여 TFLiteConverter 생성
converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/moodels/fine_tuned_model/saved_model')  # 저장된 모델 로드

# TFLite 형식으로 모델 변환
tflite_model = converter.convert()  # TFLite 모델로 변환

# 변환된 TFLite 모델을 파일로 저장
with open('/content/drive/MyDrive/models/ssd_mobilenetv2_fpnlite/detect.tflite', 'wb') as f:  # TFLite 파일 경로 지정
  f.write(tflite_model)  # 변환된 TFLite 모델 데이터를 파일에 작성

ConverterError: <unknown>:0: error: loc(callsite(callsite(fused["ConcatV2:", "Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat@__inference_call_func_11661"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_14035"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): 'tf.ConcatV2' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["ConcatV2:", "Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/ChangeCoordinateFrame/Scale/concat@__inference_call_func_11661"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_14035"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(fused["StridedSlice:", "map

# 7.&nbsp;TFLite 모델 테스트 및 평가

## 7-1. TFLite 모델을 이용한 객체 탐지 수행

In [ ]:
# 사용자 정의 TFLite 모델을 테스트 이미지에서 실행하여 객체를 탐지하는 스크립트
# 출처: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# 필요한 패키지 임포트
import os
import cv2  # 이미지 처리 라이브러리
import numpy as np
import sys
import glob  # 파일 경로를 다룰 때 사용
import random  # 테스트 이미지를 무작위로 선택
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter  # TFLite 모델 해석기

import matplotlib
import matplotlib.pyplot as plt

# Jupyter/Colab에서 이미지를 인라인으로 표시
%matplotlib inline

# TFLite 모델을 사용하여 이미지에서 객체를 탐지하고 결과를 표시하는 함수 정의
def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):
    # 테스트 폴더에서 이미지 파일 경로를 모두 가져옴
    images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

    # 라벨 맵 파일 로드
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]  # 각 라인을 클래스 이름으로 저장

    # TensorFlow Lite 모델 로드
    interpreter = Interpreter(model_path=modelpath)
    interpreter.allocate_tensors()  # 모델에서 필요한 텐서를 메모리에 할당

    # 모델 입력 및 출력 세부정보 가져오기
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]  # 입력 이미지 높이
    width = input_details[0]['shape'][2]   # 입력 이미지 너비

    float_input = (input_details[0]['dtype'] == np.float32)  # 입력 데이터 유형이 float인지 확인

    input_mean = 127.5  # 이미지 정규화를 위한 평균 값
    input_std = 127.5   # 이미지 정규화를 위한 표준 편차 값

    # 테스트 이미지 무작위 선택
    images_to_test = random.sample(images, num_test_images)

    # 각 이미지에 대해 탐지 수행
    for image_path in images_to_test:
        # 이미지 로드 및 모델 입력 크기로 리사이즈
        image = cv2.imread(image_path)  # 이미지를 읽어옴
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
        imH, imW, _ = image.shape  # 원본 이미지 크기 저장
        image_resized = cv2.resize(image_rgb, (width, height))  # 모델 입력 크기로 리사이즈
        input_data = np.expand_dims(image_resized, axis=0)  # 배치를 추가하여 [1xHxWx3] 형태로 만듦

        # 모델이 float 입력을 기대하면 정규화 수행
        if float_input:
            input_data = (np.float32(input_data) - input_mean) / input_std

        # 모델에 데이터를 입력하고 탐지 수행
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()  # 모델 실행

        # 탐지 결과 가져오기
        boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # 탐지된 객체의 바운딩 박스 좌표
        classes = interpreter.get_tensor(output_details[3]['index'])[0]  # 탐지된 객체의 클래스 인덱스
        scores = interpreter.get_tensor(output_details[0]['index'])[0]  # 탐지된 객체의 신뢰도

        detections = []  # 탐지 결과 저장

        # 탐지 결과를 바탕으로 바운딩 박스를 그리거나 결과를 저장
        for i in range(len(scores)):
            if ((scores[i] > min_conf) and (scores[i] <= 1.0)):  # 최소 신뢰도 임계값 이상인 경우만 처리
                # 바운딩 박스 좌표 계산 (이미지 크기를 고려하여 조정)
                ymin = int(max(1, (boxes[i][0] * imH)))
                xmin = int(max(1, (boxes[i][1] * imW)))
                ymax = int(min(imH, (boxes[i][2] * imH)))
                xmax = int(min(imW, (boxes[i][3] * imW)))

                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)  # 바운딩 박스 그리기

                # 객체 이름과 신뢰도 표시
                object_name = labels[int(classes[i])]  # 클래스 이름 가져오기
                label = '%s: %d%%' % (object_name, int(scores[i] * 100))  # 예: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                label_ymin = max(ymin, labelSize[1] + 10)
                cv2.rectangle(image, (xmin, label_ymin - labelSize[1] - 10), (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED)
                cv2.putText(image, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

                detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

        # 탐지된 결과를 이미지로 표시하거나 저장
        if txt_only == False:  # 이미지 결과를 표시
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 16))
            plt.imshow(image)
            plt.show()

        elif txt_only == True:  # 텍스트 파일로 저장
            image_fn = os.path.basename(image_path)
            base_fn, ext = os.path.splitext(image_fn)
            txt_result_fn = base_fn + '.txt'
            txt_savepath = os.path.join(savepath, txt_result_fn)

            # 탐지 결과를 텍스트 파일에 작성
            with open(txt_savepath, 'w') as f:
                for detection in detections:
                    f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

    return

- `PATH_TO_IMAGES` 경로 지정
- `images_to_test` 탐지 실행할 이미지 개수 지정

In [ ]:
# 사용자 모델 실행을 위한 변수 설정

# 테스트 이미지 폴더 경로
PATH_TO_IMAGES = '/content/drive/MyDrive/nipa_google/data/images_52'  # 테스트 이미지가 저장된 폴더 경로

# TFLite 모델 파일 경로
PATH_TO_MODEL = '/content/custom_model_lite/detect.tflite'  # 변환된 TFLite 모델 파일 경로

# 라벨 맵 파일 경로
PATH_TO_LABELS = '/content/labelmap.txt'  # 클래스 이름이 정의된 labelmap.txt 파일 경로

# 최소 신뢰도 임계값 설정
min_conf_threshold = 0.3  # 탐지된 객체의 신뢰도가 30% 이상인 경우만 표시
# 신뢰도가 낮은 탐지 결과도 확인하려면 값을 낮춰 예: 0.01로 변경 가능

# 테스트에 사용할 이미지 수 설정
images_to_test = 52  # 탐지를 실행할 이미지 개수

# 객체 탐지 함수 실행
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)
# TFLite 모델, 이미지 폴더, 라벨 파일, 신뢰도 임계값, 테스트 이미지 개수를 입력으로 전달

## 7-2. mAP(mean Average Precision) 계산을 통한 모델 성능 평가

In [ ]:
%%bash
# mAP(mAP - mean Average Precision) 계산을 위한 GitHub 저장소 클론
git clone https://github.com/Cartucho/mAP /content/mAP
# Cartucho의 mAP 프로젝트를 로컬로 복사하여 /content/mAP 디렉토리에 저장

# mAP 디렉토리로 이동
cd /content/mAP

# 기존 탐지 결과(detection-results) 폴더 초기화
rm input/detection-results/*

# 기존 Ground Truth(정답 데이터) 폴더 초기화
rm input/ground-truth/*

# 선택적 이미지 폴더 초기화
rm input/images-optional/*

# mAP 계산에 필요한 유틸리티 스크립트 다운로드
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py
# EdjeElectronics의 GitHub에서 mAP 계산을 도와주는 Python 스크립트를 다운로드

Cloning into '/content/mAP'...
--2024-11-27 01:59:50--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5397 (5.3K) [text/plain]
Saving to: ‘calculate_map_cartucho.py’

     0K .....                                                 100% 51.6M=0s

2024-11-27 01:59:50 (51.6 MB/s) - ‘calculate_map_cartucho.py’ saved [5397/5397]



- 아래 코드의 '이미지와 XML 파일 복사' 코드에 `PATH_TO_IMAGES` 경로 지정
  - !cp `PATH_TO_IMAGES`* /content/mAP/input/images-optional

In [ ]:
# 테스트 이미지 및 관련 XML 파일을 mAP 평가 환경으로 복사 및 이동

# 이미지와 XML 파일 복사
!cp /content/drive/MyDrive/nipa_google/data/images_52/* /content/mAP/input/images-optional
# 테스트 이미지와 관련 XML 파일을 mAP의 선택적 이미지 폴더(`images-optional`)로 복사

# XML 파일 이동
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth/
# 복사된 XML 파일을 Ground Truth 데이터 폴더(`ground-truth`)로 이동
# Ground Truth 폴더는 실제 라벨 데이터를 저장하는 폴더로, mAP 계산에 사용됨

In [ ]:
# Ground Truth XML 파일을 텍스트 파일로 변환

!python /content/mAP/scripts/extra/convert_gt_xml.py
# Pascal VOC 형식의 Ground Truth XML 파일을
# mAP 계산 도구에서 사용하는 텍스트 파일 형식으로 변환하는 스크립트를 실행

Conversion completed!


In [ ]:
# 탐지를 실행하여 결과를 텍스트 파일로 저장하기 위한 변수 설정

# 테스트 이미지 폴더 경로
PATH_TO_IMAGES = '/content/drive/MyDrive/nipa_google/data/images_52'  # 테스트 이미지가 저장된 폴더 경로

# TFLite 모델 파일 경로
PATH_TO_MODEL = '/content/custom_model_lite/detect.tflite'  # 변환된 TFLite 모델 파일 경로

# 라벨 맵 파일 경로
PATH_TO_LABELS = '/content/labelmap.txt'  # 클래스 이름이 정의된 labelmap.txt 파일 경로

# 탐지 결과를 저장할 폴더 경로
PATH_TO_RESULTS = '/content/mAP/input/detection-results'  # 탐지 결과 텍스트 파일을 저장할 디렉토리

# 최소 신뢰도 임계값 설정
min_conf_threshold = 0.1  # 탐지된 객체의 신뢰도가 10% 이상인 경우만 저장

# 테스트 폴더에 있는 모든 이미지 파일 경로를 가져오기
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list))  # 이미지가 500개 이상이면 상한선을 500개로 설정

# 탐지 결과만 텍스트 파일로 저장하도록 설정 (이미지 표시하지 않음)
txt_only = True

# 탐지 함수 실행
print('Starting inference on %d images...' % images_to_test)  # 탐지 시작 메시지 출력
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
# TFLite 모델, 이미지 폴더, 라벨 파일, 신뢰도 임계값, 테스트 이미지 수, 탐지 결과 저장 경로를 입력으로 전달
print('Finished inferencing!')  # 탐지 완료 메시지 출력

Starting inference on 52 images...
Finished inferencing!


In [ ]:
# mAP 계산을 위한 디렉토리 이동
%cd /content/mAP

# mAP(mean Average Precision) 계산 스크립트 실행
!python calculate_map_cartucho.py --labels=/content/labelmap.txt
# calculate_map_cartucho.py 스크립트를 실행하여 mAP 계산
# --labels 옵션은 라벨 맵 파일(labelmap.txt)의 경로를 지정

/content/mAP
Calculating mAP at 0.50 IoU threshold...
13.95% = bicycle AP 
20.29% = kickboard AP 
24.45% = motorcycle AP 
mAP = 19.56%
Calculating mAP at 0.55 IoU threshold...
7.10% = bicycle AP 
20.29% = kickboard AP 
22.62% = motorcycle AP 
mAP = 16.67%
Calculating mAP at 0.60 IoU threshold...
6.19% = bicycle AP 
20.29% = kickboard AP 
20.25% = motorcycle AP 
mAP = 15.58%
Calculating mAP at 0.65 IoU threshold...
5.75% = bicycle AP 
20.29% = kickboard AP 
16.72% = motorcycle AP 
mAP = 14.26%
Calculating mAP at 0.70 IoU threshold...
5.05% = bicycle AP 
14.93% = kickboard AP 
14.19% = motorcycle AP 
mAP = 11.39%
Calculating mAP at 0.75 IoU threshold...
4.47% = bicycle AP 
14.11% = kickboard AP 
8.51% = motorcycle AP 
mAP = 9.03%
Calculating mAP at 0.80 IoU threshold...
1.48% = bicycle AP 
5.58% = kickboard AP 
3.02% = motorcycle AP 
mAP = 3.36%
Calculating mAP at 0.85 IoU threshold...
0.25% = bicycle AP 
0.95% = kickboard AP 
1.07% = motorcycle AP 
mAP = 0.76%
Calculating mAP at 0.90 Io

# 8.&nbsp;TFLite 모델 배포

## 8-1. TFLite 모델 및 관련 파일 압축 및 다운로드

In [7]:
# 라벨 맵 파일과 파이프라인 구성 파일을 TFLite 모델 폴더로 이동한 후 압축하기

# 라벨 맵 파일(labelmap.txt)을 TFLite 모델 폴더로 복사
!cp /content/labelmap.txt /content/custom_model_lite

# 라벨 맵 파일(labelmap.pbtxt)을 TFLite 모델 폴더로 복사
!cp /content/labelmap.pbtxt /content/custom_model_lite

# 파이프라인 구성 파일(pipeline_file.config)을 TFLite 모델 폴더로 복사
!cp /content/models/mymodel/pipeline_file.config /content/custom_model_lite

# 현재 디렉토리를 /content로 변경
%cd /content

# TFLite 모델 폴더(custom_model_lite)를 압축하여 ZIP 파일 생성
!zip -r custom_model_lite.zip custom_model_lite
# -r 옵션: 폴더 및 하위 파일을 모두 포함하여 압축
# 결과 ZIP 파일(custom_model_lite.zip)은 /content 디렉토리에 저장

cp: cannot stat '/content/labelmap.pbtxt': No such file or directory
cp: cannot stat '/content/models/mymodel/pipeline_file.config': No such file or directory
/content
  adding: custom_model_lite (deflated 10%)


In [ ]:
from google.colab import files

# 압축된 TFLite 모델 ZIP 파일을 다운로드
files.download('/content/custom_model_lite.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 8-2. 모델 정보 확인

In [ ]:
import tensorflow as tf

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="/content/detect.tflite")
interpreter.allocate_tensors()

# 입력 정보
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1, 320, 320,   3], dtype=int32), 'shape_signature': array([  1, 320, 320,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:1', 'index': 338, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:3', 'index': 336, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameter